In [1]:
import numpy as np
import scipy
import os

In [2]:
class SignalMetrics():

    def __init__(self, pulses: np.ndarray):
        """
        Inicializa a classe com um array de pulsos.

        Args:
            pulses(ndarray): Array NumPy contendo valores dos pulsos.
        """

        if not isinstance(pulses, np.ndarray): # Validações de entrada
            raise TypeError("Pulses devem ser um array NumPy.")
        if pulses.size == 0:
            raise ValueError("Pulses não pode ser vazio.")
        
        self.pulses = pulses
        self._peak_value = None # prefixo _ indica que são variáveis internas (convenção Python)
        self._rms = None
        self._avg_amplitude = None

    @property # permite acessar o método como se fosse um atributo (sem parênteses)
    def peak_value(self) -> float:
        """Calcula e retorna o valor de pico do sinal"""
        if self._peak_value is None:
            self._peak_value = np.max(np.abs(self.pulses), axis=1)
        return self._peak_value
    
    @property
    def rms(self) -> float:
        """Calcula e retorna o valor RMS do sinal"""
        if self._rms is None:
            self._rms = np.sqrt(np.mean(self.pulses**2, axis=1))
        return self._rms
    
    @property
    def avg_amplitude(self) -> float:
        """Calcula e retorna a amplitude média de cada linha de pulsos"""
        if self._avg_amplitude is None:
            self._avg_amplitude = np.mean(np.abs(self.pulses), axis=1)
        return self._avg_amplitude
    
    @property
    def sqrt_amplitude(self) -> float:
        """Calcula e retorna a raiz quadrada da amplitude média da matriz"""
        return np.sqrt(self.avg_amplitude)

    def get_crest_factor(self) -> float:
        """Calcula e retorna o fator de crista da matriz"""
        return self.peak_value / self.rms
    
    def get_clearance_factor(self) -> float:
        """Calcula e retorna o fator de liberação da matriz"""
        return self.peak_value / self.sqrt_amplitude
    
    def impulse_factor(self) -> float:
        """Calcula e retorna o fator de impulso da matriz"""
        return self.peak_value / self.avg_amplitude
    
    def shape_factor(self) -> float:
        """Calcula e retorna o fator de forma da matriz"""
        return self.rms / self.avg_amplitude
    
    def skewness(self) -> float:
        """Calcula e retorna a assimetria da matriz"""
        return scipy.stats.skew(self.pulses, axis=1)    

    def kurtosis(self) -> float:
        """Calcula e retorna a curtose da matriz"""
        return scipy.stats.kurtosis(self.pulses, axis=1)

In [22]:
class Processor:
    def __init__(self, folder: str):
        self.folder = folder
        self.files = self.list_mat_files()

    def list_mat_files(self) -> list:
        """
        Lista arquivos .mat no diretório
        """
        files = [os.path.join(self.folder, f) for f in os.listdir(self.folder) if f.endswith('.mat')]
        return files
    
    def load_mat_data(self, file: str) -> dict:
        """
        Carrega dados de um arquivo .mat
        """
        mat_data = scipy.io.loadmat(file) # carrega arquivo como dicionário
        valid_keys = [key for key in mat_data.keys() if not key.startswith('_')] # pega chaves que não começam com '_'
        
        if not valid_keys:
            raise Exception(f'Arquivo {file} não contém dados válidos')
        return np.array(mat_data[valid_keys[0]]) # retorna o primeiro dado válido
    
    def process_single_file(self, file: str) -> np.ndarray:
        file_path = os.path.join(self.folder, file)

        if not os.path.exists(file_path):
            raise Exception(f'Arquivo {file} não encontrado')
        return self.load_mat_data(file_path)
    
    def process_files(self) -> list:
        """
        Processa todos os arquivos .mat no diretório com o algoritmo proposto
        """
        processed_files = []
        for file in self.files:
            data = self.load_mat_data(file)
            data_pos, data_neg = self.half_cycle(data)
            processed_files.append((data_pos, data_neg))   
        return processed_files
    
    def normalization(self, data):
        # Normalização dos dados
        data[2:] = data[2:] / data[0]
        return data
    
    def half_cycle(self, data):
        data = self.normalization(data)
        # Separação dos dados em semiciclo positivo e negativo
        classes = np.where(data[1, :] <= 180, 'positivo', 'negativo') # insere classes para filtragem
        data_w_classes = np.vstack((data, classes))
        # Seleção correta de índices onde os rótulos são positivos
        mask_positivo = data_w_classes[-1] == 'positivo'
        mask_negativo = ~mask_positivo # tudo que não é positivo é negativo
        data_pos = data_w_classes[:, mask_positivo]
        data_neg = data_w_classes[:, mask_negativo]
        # Deletar classes 
        data_pos = np.delete(data_pos, -1, axis=0).astype(np.float64) if data_pos.size > 0 else np.array([]) # evita erros de conversão se o array estiver vazio
        data_neg = np.delete(data_neg, -1, axis=0).astype(np.float64) if data_neg.size > 0 else np.array([]) 
        return data_pos, data_neg



In [ ]:
folder_path = r"C:\Users\jmlnn\OneDrive\Engenharia\0. Pesquisa DP\arquivos"
processor = Processor(folder_path)
mat_files = processor.list_mat_files()

processed_files = processor.process_files()
print(processed_files[0][0]) # primeiro indice: arquivo, segundo indice: semiciclo (0 = positivo, 1 = negativo)


[[ 1.26266406e-01  1.07125781e-01  9.69695313e-02 ...  1.19235156e-01
   1.05172656e-01  1.18453906e-01]
 [ 2.42780885e+01  4.71812595e+01  7.35990319e+01 ...  2.82599990e+01
   4.89916281e+01  7.63670467e+01]
 [ 1.40684932e-01  2.46507735e-01  2.59306383e-01 ...  1.44028488e-01
   1.85119158e-01  2.06569011e-02]
 ...
 [ 1.40684932e-01  1.10353228e-01  2.80077480e-01 ...  1.57766421e-01
   6.15180416e-02  1.18095113e-01]
 [ 1.52935448e-01 -8.78196545e-03  7.23665043e-02 ...  1.16552622e-01
   9.68326463e-02  9.02556238e-02]
 [ 1.40684932e-01  4.22759746e-02  1.76221992e-01 ...  6.16008910e-02
   2.55748368e-01  1.32014857e-01]]


In [ ]:
data_pos, data_neg = processed_files[0] # semiciclos positivo e negativo do primeiro arquivo

In [29]:
# Separação dos dados de treino e teste
from sklearn.model_selection import train_test_split
X_train_pos, X_test_pos = train_test_split(data_pos, test_size=0.2, random_state=42)
X_train_neg, X_test_neg = train_test_split(data_neg, test_size=0.2, random_state=42)

In [30]:
metrics_sp = SignalMetrics(X_train_pos)

X_pos_features = np.array([
    metrics_sp.avg_amplitude,
    metrics_sp.rms,
    metrics_sp.peak_value,
    metrics_sp.sqrt_amplitude,
    metrics_sp.get_crest_factor(),
    metrics_sp.get_clearance_factor(),
    metrics_sp.impulse_factor(),
    metrics_sp.shape_factor(),
    metrics_sp.skewness(),
    metrics_sp.kurtosis()
])

In [31]:
metrics_sp_test = SignalMetrics(X_test_pos)

features_sp_test = np.array([
    metrics_sp_test.avg_amplitude,
    metrics_sp_test.rms,
    metrics_sp_test.peak_value,
    metrics_sp_test.sqrt_amplitude,
    metrics_sp_test.get_crest_factor(),
    metrics_sp_test.get_clearance_factor(),
    metrics_sp_test.impulse_factor(),
    metrics_sp_test.shape_factor(),
    metrics_sp_test.skewness(),
    metrics_sp_test.kurtosis()
])

In [33]:
metrics_sn = SignalMetrics(X_train_neg)

features_sn_train = np.array([
    metrics_sn.avg_amplitude,
    metrics_sn.rms,
    metrics_sn.peak_value,
    metrics_sn.sqrt_amplitude,
    metrics_sn.get_crest_factor(),
    metrics_sn.get_clearance_factor(),
    metrics_sn.impulse_factor(),
    metrics_sn.shape_factor(),
    metrics_sn.skewness(),
    metrics_sn.kurtosis()
])

In [34]:
metrics_sn_test = SignalMetrics(X_test_neg)

features_sn_test = np.array([
    metrics_sn_test.avg_amplitude,
    metrics_sn_test.rms,
    metrics_sn_test.peak_value,
    metrics_sn_test.sqrt_amplitude,
    metrics_sn_test.get_crest_factor(),
    metrics_sn_test.get_clearance_factor(),
    metrics_sn_test.impulse_factor(),
    metrics_sn_test.shape_factor(),
    metrics_sn_test.skewness(),
    metrics_sn_test.kurtosis()
])

In [35]:
print(f"features_sp_train.shape: {X_train_pos.shape}")
print(f"features_sp_test.shape: {X_test_pos.shape}")
print(f"features_sn_train.shape: {X_train_neg.shape}")
print(f"features_sn_train.shape: {X_test_neg.shape}")


features_sp_train.shape: (322, 29)
features_sp_test.shape: (81, 29)
features_sn_train.shape: (322, 23)
features_sn_train.shape: (81, 23)
